In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '../../bar_movement/data/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Gbp_Jpy_M5_2021-2022.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

df_small = pd.read_csv(file_path + 'Oanda_Gbp_Jpy_M1_2021-2022.csv')
df_small.Date = pd.to_datetime(df_small.Date)
df_small.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2021-01-03 22:00:00   140.724   140.953  140.693    140.862   140.874   
1 2021-01-03 22:05:00   140.874   140.896  140.842    140.852   141.024   
2 2021-01-03 22:10:00   140.860   140.866  140.854    140.866   141.007   
3 2021-01-03 22:15:00   140.861   140.861  140.861    140.861   140.998   
4 2021-01-03 22:20:00   140.871   140.999  140.871    140.923   141.000   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  \
0   141.103  140.843    141.012   140.799   141.028  140.768    140.937   
1   141.046  140.992    141.002   140.949   140.971  140.917    140.927   
2   141.016  140.998    141.003   140.934   140.941  140.929    140.934   
3   140.998  140.998    140.998   140.930   140.930  140.930    140.930   
4   141.126  141.000    141.073   140.936   141.061  140.936    140.998   

   Volume  
0      53  
1      27  
2       7  
3       1  
4      50  
--------------------------

In [5]:
print(df_small.head())
print('-' * 75)
print(df_small.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2021-04-01 06:00:00   152.407   152.409  152.375    152.385   152.431   
1 2021-04-01 06:01:00   152.387   152.421  152.380    152.401   152.411   
2 2021-04-01 06:02:00   152.404   152.409  152.383    152.392   152.428   
3 2021-04-01 06:03:00   152.390   152.416  152.380    152.414   152.413   
4 2021-04-01 06:04:00   152.410   152.411  152.397    152.401   152.435   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  \
0   152.435  152.399    152.409   152.419   152.422  152.388    152.397   
1   152.443  152.405    152.427   152.399   152.432  152.392    152.414   
2   152.432  152.406    152.415   152.416   152.420  152.394    152.404   
3   152.440  152.403    152.438   152.402   152.428  152.392    152.426   
4   152.435  152.421    152.425   152.422   152.422  152.409    152.413   

   Volume  
0      49  
1      56  
2      51  
3      32  
4      38  
--------------------------

In [6]:
def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(barsdata, lookback=14):
    high_low = barsdata['Mid_High'] - barsdata['Mid_Low']
    high_close = np.abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    low_close = np.abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback


def rsi(barsdata, periods=14):
    close_delta = barsdata['Mid_Close'].diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def stoch_rsi(data, k_window=3, d_window=3, window=14):
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()

    stoch = ((data - min_val) / (max_val - min_val)) * 100

    slow_k = stoch.rolling(window=k_window, center=False).mean()

    slow_d = slow_k.rolling(window=d_window, center=False).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def chop(df, lookback=14):
    atr1 = atr(df, lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

def bar_lengths(bar_lens, window=36):
    return bar_lens.rolling(window=window).mean(), bar_lens.rolling(window=window).std()

def sar_lengths(opens, sars, window=36):
    diffs = abs(opens - sars.shift(1))

    return diffs.rolling(window=window).mean(), diffs.rolling(window=window).std()

def range_high_lows(df, i, n_histograms=6):
    green, red = False, False
    curr_color, curr_streak = None, 0
    high, low = -np.inf, np.inf

    while i >= 0:
        hist = df.loc[df.index[i], 'macdhist']
        curr_high, curr_low = df.loc[df.index[i], ['Mid_High', 'Mid_Low']]
        new_color = 'green' if hist > 0 else 'red'

        if curr_color != new_color:
            curr_color = new_color
            curr_streak = 0

            if green and red:
                break

        curr_streak += 1

        if curr_streak >= n_histograms:
            high = max(high, curr_high)
            low = min(low, curr_low)

            if curr_color == 'red':
                red = True

            else:
                green = True

        i -= 1

    return high, low if (high > -np.inf and low < np.inf) else np.nan, np.nan

# def range_high_lows(df, i):
#     green, red = False, False
#     curr_color, curr_streak = None, 0
#     high, low = -np.inf, np.inf

#     while i >= 0:
#         hist = df.loc[df.index[i], 'macdhist']
#         curr_high, curr_low = df.loc[df.index[i], ['Mid_High', 'Mid_Low']]
#         new_color = 'green' if hist > 0 else 'red'

#         if curr_color != new_color:
#             curr_color = new_color
#             curr_streak = 0

#             if green and red:
#                 break

#         curr_streak += 1

#         if curr_streak >= 3:
#             if curr_color == 'red':
#                 low = min(low, curr_low)
#                 red = True

#             else:
#                 high = max(high, curr_high)
#                 green = True

#         i -= 1

#     return high, low if (high > -np.inf and low < np.inf) else np.nan, np.nan


In [7]:
# Add technical indicators (for additional features)
df['atr'] = atr(df)
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df)

                     Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0     2021-01-03 23:10:00   141.043   141.101  141.020    141.094   141.105   
1     2021-01-03 23:15:00   141.092   141.125  141.072    141.093   141.141   
2     2021-01-03 23:20:00   141.092   141.144  141.067    141.128   141.122   
3     2021-01-03 23:25:00   141.131   141.198  141.121    141.179   141.164   
4     2021-01-03 23:30:00   141.178   141.209  141.154    141.199   141.217   
...                   ...       ...       ...      ...        ...       ...   
74762 2021-12-31 21:35:00   155.595   155.620  155.583    155.620   155.642   
74763 2021-12-31 21:40:00   155.617   155.631  155.609    155.611   155.658   
74764 2021-12-31 21:45:00   155.605   155.634  155.601    155.629   155.659   
74765 2021-12-31 21:50:00   155.631   155.692  155.613    155.638   155.692   
74766 2021-12-31 21:55:00   155.642   155.689  155.614    155.659   155.720   

       Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_

In [8]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]
pair = 'EUR_USD'

In [9]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(open_price, stop_loss, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = abs(open_price - stop_loss)
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'JPY' else pips_to_risk * 100

    if second == 'USD':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'JPY' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, atr_multiplier, n_minutes, adx_cutoff):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    end_date = None
    limit_order = None

    def iterate_thru_small_data(lo):
        def get_end_date(j):
            return df_small.loc[df_small.index[j + 1], 'Date'] if j + 1 < len(df_small) else None

        filtered = df_small[df_small.Date >= lo['start_date']]

        if len(filtered) == 0:
            return -1

        j = filtered.index[0]

        nonlocal reward
        nonlocal day_fees
        nonlocal n_losses
        nonlocal n_wins
        nonlocal n_buys
        nonlocal n_sells
        nonlocal curr_win_streak
        nonlocal curr_loss_streak
        nonlocal win_streak
        nonlocal loss_streak
        nonlocal pips_risked
        trade = None

        while j < len(df_small):
            curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, cur_date = df_small.loc[df_small.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close', 'Date']]

            if lo is not None:
                minutes_elapsed = (cur_date - lo['start_date']).total_seconds() / 60
                curr_long = df[df.Date < cur_date]
                adx_long = curr_long.loc[curr_long.index[-1], 'adx']

                if minutes_elapsed >= n_minutes or adx_long >= adx_cutoff:
                    return cur_date

            if trade is None and lo is not None:
                place_buy = curr_ask_low <= lo['buy_open_price']
                place_sell = curr_bid_high >= lo['sell_open_price']

                if place_buy:
                    open_price = lo['buy_open_price']
                    stop_gain = lo['buy_stop_gain']
                    pips_gained = stop_gain - open_price
                    curr_pips_to_risk = pips_gained / risk_reward_ratio
                    stop_loss = lo['buy_stop_loss']

                    if stop_loss < open_price and stop_gain > open_price:
                        n_units = lo['buy_n_units']

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': cur_date, 'end_date': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)
                        
                        lo = None

                elif place_sell:
                    open_price = lo['sell_open_price']
                    stop_gain = lo['sell_stop_gain']
                    pips_gained = open_price - stop_gain
                    curr_pips_to_risk = pips_gained / risk_reward_ratio
                    stop_loss = lo['sell_stop_loss']

                    if stop_loss > open_price and stop_gain < open_price:
                        n_units = lo['sell_n_units']

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                'n_units': n_units, 'original_units': n_units, 'start_date': cur_date, 'end_date': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)

                        lo = None

            if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
                trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], cur_date, trade['n_units'])

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > win_streak:
                    win_streak = curr_win_streak

                if curr_loss_streak > loss_streak:
                    loss_streak = curr_loss_streak

                break

            if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
                trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], cur_date, trade['n_units'])

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > win_streak:
                    win_streak = curr_win_streak

                if curr_loss_streak > loss_streak:
                    loss_streak = curr_loss_streak

                break

            if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
                trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], cur_date, trade['n_units'])

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > win_streak:
                    win_streak = curr_win_streak

                if curr_loss_streak > loss_streak:
                    loss_streak = curr_loss_streak

                break

            if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
                trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], cur_date, trade['n_units'])

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > win_streak:
                    win_streak = curr_win_streak

                if curr_loss_streak > loss_streak:
                    loss_streak = curr_loss_streak

                break

            j += 1

        return cur_date

    for i in range(1, len(df)):
        if end_date == -1:
            break

        curr_date = df.loc[df.index[i], 'Date']

        if end_date is not None and end_date >= curr_date:
            continue

        end_date = None
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_mid_open, curr_mid_low = \
            df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_Low']]
        atr, adx, ask_low, bid_high = df.loc[df.index[i - 1], ['atr', 'adx', 'Ask_Low', 'Bid_High']]
        chop_signal = adx < adx_cutoff

        if limit_order is None and chop_signal:
            range_high, range_low = bid_high + (atr * atr_multiplier), ask_low - (atr * atr_multiplier)
            rang = abs(float(range_high) - float(range_low))
            pips_gain = (rang / 2) * risk_reward_ratio
            pips_to_risk = pips_gain / risk_reward_ratio
            
            if range_high > range_low and spread <= pips_to_risk * spread_cutoff:
                buy_open_price = float(range_low)
                buy_stop_gain = buy_open_price + (rang / 2) * risk_reward_ratio
                buy_pips_gained = buy_stop_gain - buy_open_price
                buy_pips_to_risk = buy_pips_gained / risk_reward_ratio
                buy_pullback = buy_open_price - buy_pips_to_risk
                buy_stop_loss = round(buy_pullback, 5)
                buy_mo = buy_open_price - (spread / 2)
                buy_n_units = get_n_units(buy_open_price, buy_stop_loss, buy_mo, pair)

                sell_open_price = float(range_high)
                sell_stop_gain = sell_open_price - (rang / 2) * risk_reward_ratio
                sell_pips_gained = sell_open_price - sell_stop_gain
                sell_pips_to_risk = sell_pips_gained / risk_reward_ratio
                sell_pullback = sell_open_price + sell_pips_to_risk
                sell_stop_loss = round(sell_pullback, 5)
                sell_mo = sell_open_price + (spread / 2)
                sell_n_units = get_n_units(sell_open_price, sell_stop_loss, sell_mo, pair)

                limit_order = {'buy_open_price': buy_open_price, 'buy_stop_gain': buy_stop_gain, 'buy_stop_loss': buy_stop_loss, 'buy_n_units': buy_n_units, 
                               'sell_open_price': sell_open_price, 'sell_stop_gain': sell_stop_gain, 'sell_stop_loss': sell_stop_loss, 'sell_n_units': sell_n_units, 'start_date': curr_date}

        if limit_order is not None:
            end_date = iterate_thru_small_data(limit_order)
            limit_order = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [11]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
# risk_reward_ratio_vals = [0.25, 0.50, 1.0]
# spread_cutoffs = [0.10, 0.20]
# atr_multipliers = [0.5, 1.0]  # Determines how wide to make the sideways range
# n_minutes_vals = [10, 15, 20, 30]
# adx_cutoffs = [25, 30]

risk_reward_ratio_vals = [0.5]
spread_cutoffs = [0.10]
atr_multipliers = [1.0]
n_minutes_vals = [10]
adx_cutoffs = [25]

all_combos = []


# use_half_sar, vo_cutoff, use_candles

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for atr_multiplier in atr_multipliers:
            for n_minutes in n_minutes_vals:
                for adx_cutoff in adx_cutoffs:
                    all_combos.append((risk_reward_ratio, spread_val, atr_multiplier, n_minutes, adx_cutoff))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_atr_multiplier = None
best_n_minutes= None
best_adx_cutoff = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, atr_multiplier, n_minutes, adx_cutoff in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, atr_multiplier, n_minutes, adx_cutoff)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    # print('Avg pips risked: ' + str(sum(pips_risked) / len(pips_risked)))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread_val': spread_val, 'atr_multiplier': atr_multiplier, 'n_minutes': n_minutes, 'adx_cutoff': adx_cutoff})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_atr_multiplier = atr_multiplier
        best_n_minutes = n_minutes
        best_adx_cutoff = adx_cutoff

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 1

24.80043000000498
Num buys: 2
Num sells: 5
Num trades: 7
Num wins: 5
Num losses: 2
Win streak: 3
Loss streak: 1
Remaining runs: 0
Best reward so far: 24.80043000000498



In [12]:
# EUR/USD
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n minutes: ' + str(best_n_minutes))
print('Best adx cutoff: ' + str(best_adx_cutoff))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 546.317235714308
Best risk/reward ratio: 0.5
Best spread: 0.1
Best atr multiplier: 1.0
Best n minutes: 10
Best adx cutoff: 30
-----------------------
Top results:
{'reward': 373, 'ratio': 0.5, 'spread_val': 0.1, 'atr_multiplier': 1.0, 'n_minutes': 10, 'adx_cutoff': 25}
{'reward': 212, 'ratio': 0.25, 'spread_val': 0.1, 'atr_multiplier': 0.5, 'n_minutes': 15, 'adx_cutoff': 30}
{'reward': 274, 'ratio': 0.25, 'spread_val': 0.1, 'atr_multiplier': 0.5, 'n_minutes': 10, 'adx_cutoff': 30}
{'reward': 187, 'ratio': 0.25, 'spread_val': 0.1, 'atr_multiplier': 0.5, 'n_minutes': 15, 'adx_cutoff': 25}
{'reward': 169, 'ratio': 0.5, 'spread_val': 0.1, 'atr_multiplier': 1.0, 'n_minutes': 20, 'adx_cutoff': 25}
{'reward': 342, 'ratio': 0.5, 'spread_val': 0.1, 'atr_multiplier': 1.0, 'n_minutes': 20, 'adx_cutoff': 30}
{'reward': 137, 'ratio': 0.25, 'spread_val': 0.1, 'atr_multiplier': 0.5, 'n_minutes': 20, 'adx_cutoff': 25}
{'reward': 170, 'ratio': 0.5, '

In [13]:
# GBP/USD
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n minutes: ' + str(best_n_minutes))
print('Best adx cutoff: ' + str(best_adx_cutoff))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 398.52514642858495
Best risk/reward ratio: 0.5
Best spread: 0.1
Best atr multiplier: 1.5
Best n minutes: 15
Best adx cutoff: 25
-----------------------
Top results:
{'reward': -166, 'ratio': 1.0, 'spread_val': 0.1, 'atr_multiplier': 1.5, 'n_minutes': 10, 'adx_cutoff': 25}
{'reward': -107, 'ratio': 1.0, 'spread_val': 0.1, 'atr_multiplier': 1.0, 'n_minutes': 15, 'adx_cutoff': 20}
{'reward': 212, 'ratio': 0.25, 'spread_val': 0.1, 'atr_multiplier': 1.5, 'n_minutes': 15, 'adx_cutoff': 20}
{'reward': 75, 'ratio': 0.5, 'spread_val': 0.1, 'atr_multiplier': 1.0, 'n_minutes': 15, 'adx_cutoff': 25}
{'reward': -124, 'ratio': 0.5, 'spread_val': 0.1, 'atr_multiplier': 1.5, 'n_minutes': 10, 'adx_cutoff': 20}
{'reward': 162, 'ratio': 0.25, 'spread_val': 0.1, 'atr_multiplier': 1.0, 'n_minutes': 10, 'adx_cutoff': 25}
{'reward': 348, 'ratio': 0.5, 'spread_val': 0.1, 'atr_multiplier': 1.5, 'n_minutes': 15, 'adx_cutoff': 20}
{'reward': 24, 'ratio': 0.5, 

In [10]:
# AUD/USD
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n bars: ' + str(best_n_bars))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1876.0760714286596
Best risk/reward ratio: 0.5
Best spread: 0.2
Best atr multiplier: 1.0
Best n bars: 2
-----------------------
Top results:
{'reward': 95, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 2}
{'reward': 1798, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 1}
{'reward': 1876, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 2}
{'reward': 1650, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 3}
{'reward': 1700, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 1}
{'reward': 1497, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 2}
{'reward': 50, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 3}
{'reward': 850, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 3}
{'reward': 1199, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 1}


In [12]:
# USD/CAD
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n bars: ' + str(best_n_bars))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 4048.6535253571433
Best risk/reward ratio: 0.5
Best spread: 0.2
Best atr multiplier: 1.0
Best n bars: 1
-----------------------
Top results:
{'reward': 2053, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 2}
{'reward': 716, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 2}
{'reward': 4048, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 1}
{'reward': 636, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 3}
{'reward': 523, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 3}
{'reward': 2977, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 3}
{'reward': 3480, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 2}
{'reward': 2039, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 1}
{'reward': 2445, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 1}


In [11]:
# USD/JPY
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n bars: ' + str(best_n_bars))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 250758.18965857464
Best risk/reward ratio: 0.5
Best spread: 0.2
Best atr multiplier: 1.0
Best n bars: 1
-----------------------
Top results:
{'reward': 145619, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 3}
{'reward': 192316, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 2}
{'reward': 37162, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 3}
{'reward': 176101, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 2}
{'reward': 225827, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 1}
{'reward': 250758, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 1}
{'reward': 206984, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 2}
{'reward': 135342, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 1}
{'reward': 148785, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 3}


In [10]:
# USD/CHF
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n bars: ' + str(best_n_bars))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1456.2561146429143
Best risk/reward ratio: 0.5
Best spread: 0.2
Best atr multiplier: 1.5
Best n bars: 2
-----------------------
Top results:
{'reward': 1456, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 2}
{'reward': 1316, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 1}
{'reward': 225, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 3}
{'reward': 1034, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 1}
{'reward': 207, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 3}
{'reward': 690, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 1}
{'reward': 296, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 2}
{'reward': 302, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 2}
{'reward': 1339, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 3}


In [10]:
# NZD/USD
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n bars: ' + str(best_n_bars))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1074.2768450000149
Best risk/reward ratio: 0.5
Best spread: 0.2
Best atr multiplier: 1.0
Best n bars: 3
-----------------------
Top results:
{'reward': 725, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 1}
{'reward': 1074, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 3}
{'reward': -25, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 3}
{'reward': 624, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 2}
{'reward': 750, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 2}
{'reward': 449, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 1}
{'reward': 549, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 1}
{'reward': 774, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 3}
{'reward': 1049, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 2}


In [11]:
# GBP/CHF
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n minutes: ' + str(best_n_minutes))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 259.48194125004545
Best risk/reward ratio: 1.5
Best spread: 0.2
Best atr multiplier: 1.5
Best n minutes: 5
-----------------------
Top results:
{'reward': 259, 'ratio': 1.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_minutes': 5}
{'reward': -282, 'ratio': 1.5, 'spread_val': 0.1, 'atr_multiplier': 1.5, 'n_minutes': 10}
{'reward': -1, 'ratio': 0.5, 'spread_val': 0.1, 'atr_multiplier': 1.0, 'n_minutes': 5}
{'reward': 0, 'ratio': 1.0, 'spread_val': 0.1, 'atr_multiplier': 1.0, 'n_minutes': 5}
{'reward': -51, 'ratio': 1.5, 'spread_val': 0.1, 'atr_multiplier': 1.5, 'n_minutes': 5}
{'reward': 0, 'ratio': 0.5, 'spread_val': 0.1, 'atr_multiplier': 1.0, 'n_minutes': 10}
{'reward': -24, 'ratio': 0.5, 'spread_val': 0.3, 'atr_multiplier': 2.0, 'n_minutes': 5}
{'reward': -105, 'ratio': 0.5, 'spread_val': 0.1, 'atr_multiplier': 2.0, 'n_minutes': 10}
{'reward': -126, 'ratio': 0.5, 'spread_val': 0.1, 'atr_multiplier': 1.5, 'n_minutes': 10}
{'reward'

In [10]:
# EUR/JPY
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n bars: ' + str(best_n_bars))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 403452.127566436
Best risk/reward ratio: 0.5
Best spread: 0.2
Best atr multiplier: 1.5
Best n bars: 2
-----------------------
Top results:
{'reward': 222544, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 1}
{'reward': 127405, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 2}
{'reward': 285747, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 1}
{'reward': 171298, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 2}
{'reward': 199570, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 3}
{'reward': 403452, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_bars': 2}
{'reward': 178521, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 1}
{'reward': 100253, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 1.0, 'n_bars': 3}
{'reward': 30516, 'ratio': 0.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_bars': 3}
